# Notebook 4: Method Comparison and Insights

This notebook compares Cell2location results against RCTD, SPOTlight, and Tangram.
We evaluate agreement, biological plausibility, and interpretability.

In [1]:
# Install Tangram (others require R)
!pip install tangram-sc --quiet

ERROR! Session/line number was not unique in database. History logging moved to new session 6


In [2]:
# Import libraries
import scanpy as sc
import tangram as tg
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
# Load data
adata_sc = sc.read('../data/scRNA_seq/GSE194247/adata_sc_preprocessed.h5ad')
adata_sp = sc.read('../results/cell_abundances/adata_sp_cell2location.h5ad')

In [ ]:
# Prepare data for Tangram
tg.pp_adatas(adata_sc, adata_sp, genes=None)

In [ ]:
# Train Tangram model (sc -> spatial)
tmap = tg.map_cells_to_space(
    adata_sc, adata_sp,
    mode='cells', density_prior='uniform', device='cpu', num_epochs=100
)

In [ ]:
# Project cell type annotations back to space
pred = tg.project_cell_annotations(
    tmap, annotation='cell_type', adata_sp=adata_sp
)
for k in pred.var_names:
    adata_sp.obs[f'Tangram_{k}'] = pred[:, k].X.flatten()

In [ ]:
# Compare cell2location vs Tangram for example cell type
sns.scatterplot(
    x=adata_sp.obs['Cancer_Classical'],
    y=adata_sp.obs['Tangram_Cancer_Classical']
)
plt.xlabel('Cell2location')
plt.ylabel('Tangram')
plt.title('Cancer_Classical cell type abundance')
plt.show()

**Notes**:
- RCTD and SPOTlight should be run in R separately.
- Store RCTD/SPOTlight results as `.csv` and import for comparison.
- Use correlation and visual overlay to assess agreement.

In [ ]:
# Example correlation matrix
cell2loc = adata_sp.obsm['means_cell_abundance_w_sf']
tangram = adata_sp.obs[[k for k in adata_sp.obs.columns if k.startswith('Tangram_')]]
corr_df = pd.DataFrame(np.corrcoef(cell2loc.T, tangram.T)[:cell2loc.shape[1], cell2loc.shape[1]:],
                      index=cell2loc.columns, columns=tangram.columns)
sns.heatmap(corr_df, cmap='vlag')
plt.title('Correlation: Cell2location vs Tangram')
plt.show()